In [16]:
import open3d as o3d

In [17]:
# load the meshes
import numpy as np
# mesh path
from dsd import DATA_DIR

mug_mesh_path = DATA_DIR / "meshes" / "shoes" / "GSO-shoes"
meshes = list(mug_mesh_path.glob("**/*.obj"))
print(meshes)

[PosixPath('/home/tlips/Documents/diffusing-synthetic-data/data/meshes/shoes/GSO-shoes/GSO/Adrenaline_GTS_13_Color_DrkDenimWhtBachlorBttnSlvr_Size_50_yfK40TNjq0V.obj'), PosixPath('/home/tlips/Documents/diffusing-synthetic-data/data/meshes/shoes/GSO-shoes/GSO/Womens_Canvas_Bahama_in_White_UfZPHGQpvz0.obj'), PosixPath('/home/tlips/Documents/diffusing-synthetic-data/data/meshes/shoes/GSO-shoes/GSO/Timberland_Womens_Classic_Amherst_2Eye_Boat_Shoe.obj'), PosixPath('/home/tlips/Documents/diffusing-synthetic-data/data/meshes/shoes/GSO-shoes/GSO/TERREX_FAST_R.obj'), PosixPath('/home/tlips/Documents/diffusing-synthetic-data/data/meshes/shoes/GSO-shoes/GSO/PureConnect_2_Color_FernNightlifeSilverBlack_Size_70_5w0BYsiogeV.obj'), PosixPath('/home/tlips/Documents/diffusing-synthetic-data/data/meshes/shoes/GSO-shoes/GSO/UGG_Bailey_Button_Womens_Boots_Black_7.obj'), PosixPath('/home/tlips/Documents/diffusing-synthetic-data/data/meshes/shoes/GSO-shoes/GSO/Real_Deal_1nIwCHX1MTh.obj'), PosixPath('/home/t

In [33]:
# load first mesh
i = 38
mesh = o3d.io.read_triangle_mesh(str(meshes[i]))

# simplify the mesh
mesh = mesh.simplify_quadric_decimation(2000)


# get the principal axis of the z-projected mesh

vertices = np.array(mesh.vertices)

xy_vertices = vertices[:, :2]

# do pca on the xy vertices
cov = np.cov(xy_vertices.T)
eig_vals, eig_vecs = np.linalg.eig(cov)

# get the principal axis
principal_xy_axis = eig_vecs[:, np.argmax(eig_vals)]
print(principal_xy_axis)



# visualize principal axis as line 
line = o3d.geometry.LineSet()

mean = np.mean(xy_vertices, axis=0)
helper = mean + principal_xy_axis

# make 3d points
mean = np.append(mean, 0)
helper = np.append(helper, 0)

line.points = o3d.utility.Vector3dVector([mean, helper])
line.lines = o3d.utility.Vector2iVector([[0, 1]])


# find the outer points on that line in the mesh

# project the vertices on the principal axis
principal_axis = np.append(principal_xy_axis, 0)

# check if the axis points from toe to heel by checking if the z component at the toe is higher than at the bottom
vertices = np.array(mesh.vertices)
vertices_2d = vertices[:, :2]


# only take vertices which are close to the principal axis
vertices = np.array(mesh.vertices)


vertices_2d = vertices[:, :2]
vertices_2d = vertices_2d/ np.linalg.norm(vertices_2d, axis=1)[:, None]
projected_vertices = np.dot(vertices_2d, principal_xy_axis)
# get vertices with angle less than 30 degrees
vertices = vertices[np.where(np.abs(projected_vertices) > 0.99)]
print(len(vertices))

z_selected_vertices = vertices[np.where(vertices[:, 2] < 0.04)]
print(len(z_selected_vertices))
projected_vertices = np.dot(z_selected_vertices, principal_axis)
heel_idx = np.argmin(projected_vertices)
heel_point = z_selected_vertices[heel_idx]



z_selected_vertices = vertices[np.where(vertices[:, 2] < 0.06)]
projected_vertices = np.dot(z_selected_vertices, principal_axis)
toe_idx = np.argmax(projected_vertices)
toe_point = z_selected_vertices[toe_idx]


# get the heel direction
heel_direction = toe_point - heel_point

filtered_points = vertices[np.where(vertices[:, 0] > heel_point[0] + 0.1 *principal_axis[0])]

# get coordinates of points on principal frame



top_point = filtered_points[np.argmax(filtered_points[:, 2])]



# visualize the min and max points
heel_point_o3d = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)
heel_point_o3d.translate(heel_point)
heel_point_o3d.paint_uniform_color([1, 0, 0])

toe_point_o3d = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)
toe_point_o3d.translate(toe_point)
toe_point_o3d.paint_uniform_color([0, 1, 0])

top_point_o3d = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)
top_point_o3d.translate(top_point)
top_point_o3d.paint_uniform_color([0, 0, 1])



# visualize the mesh
o3d.visualization.draw_geometries([mesh, line, heel_point_o3d, toe_point_o3d, top_point_o3d])

[Open3D WARNING] [SimplifyQuadricDecimation] This mesh contains triangle uvs that are not handled in this function
[-0.13636641 -0.99065847]
210
100
